# W4111: Introduction to Databases - NoSQL  (Neo4j) HW4 
## UNI: sh3907
### Part 1- Neo4j

### 1. FanGraph Class Implementation

In [1]:
from py2neo import data, Graph, NodeMatcher, Node, Relationship, RelationshipMatcher

import json
import uuid


class FanGraph(object):
    """
    This object provides a set of helper methods for creating and retrieving nodes and relationships from
    a Neo4j database holding information about players, teams, fans, comments and their relationships.
    """

    # Note:
    # I tend to avoid object mapping frameworks. Object mapping frameworks are fun in the beginning
    # but tend to be annoying after a while. So, I did not create types Player, Team, etc.
    #

    # Connects to the DB and sets a Graph instance variable.
    # Also creates a NodeMatcher and RelationshipMatcher, which are a py2neo framework classes.
    def __init__(self, auth=('neo4j', 'sh3907'), host='localhost', port=7687, secure=False, ):
        self._graph = Graph(secure=secure,
                            bolt=True,
                            auth=auth,
                            host=host,
                            port=port)
        self._node_matcher = NodeMatcher(self._graph)
        self._relationship_matcher = RelationshipMatcher(self._graph)

    def run_q(self, qs, args):
        """

        :param qs: Query string that may have {} slots for parameters.
        :param args: Dictionary of parameters to insert into query string.
        :return:  Result of the query, which executes as a single, standalone transaction.
        """
        try:
            tx = self._graph.begin(autocommit=False)
            result = self._graph.run(qs, args)
            return result
        except Exception as e:
            print("Run exaception = ", e)

    def run_match(self, labels=None, properties=None):
        """
        Uses a NodeMatcher to find a node matching a "template."
        :param labels: A list of labels that the node must have.
        :param properties: A dictionary of {property_name: property_value} defining the template that the
            node must match.
        :return: An array of Node objects matching the pattern.
        """
        # ut.debug_message("Labels = ", labels)
        # ut.debug_message("Properties = ", json.dumps(properties))

        if labels is not None and properties is not None:
            result = self._node_matcher.match(labels, **properties)
        elif labels is not None and properties is None:
            result = self._node_matcher.match(labels)
        elif labels is None and properties is not None:
            result = self._node_matcher.match(**properties)
        else:
            raise ValueError("Invalid request. Labels and properties cannot both be None.")

        # Convert NodeMatch data into a simple list of Nodes.
        full_result = []
        for r in result:
            full_result.append(r)

        return full_result

    def find_nodes_by_template(self, tmp):
        """

        :param tmp: A template defining the label and properties for Nodes to return. An
         example is { "label": "Fan", "template" { "last_name": "Ferguson", "first_name": "Donald" }}
        :return: A list of Nodes matching the template.
        """
        labels = tmp.get('label', None)
        props = tmp.get("template", None)
        result = self.run_match(labels=labels, properties=props)
        return result

    # Create and save a new node for  a 'Fan.'
    def create_fan(self, uni, last_name, first_name):
        """

        :param uni: uni
        :param last_name: Obvious
        :param first_name: Obvious
        :return: Node created.

        NOTE: This does not check uni uniqueness. We could do using transactions or setting a constraint
        on the database.
        """
        n = Node("Fan", uni=uni, last_name=last_name, first_name=first_name)
        tx = self._graph.begin(autocommit=True)
        tx.create(n)
        return n

    # Given a UNI, return the node for the Fan.
    def get_fan(self, uni):
        n = self.find_nodes_by_template({"label": "Fan", "template": {"uni": uni}})
        if n is not None and len(n) > 0:
            # I should throw an exception here if there is more than 1.
            n = n[0]
        else:
            n = None

        return n

    def create_player(self, player_id, last_name, first_name):
        n = Node("Player", player_id=player_id, last_name=last_name, first_name=first_name)
        tx = self._graph.begin(autocommit=True)
        tx.create(n)
        return n

    def get_player(self, player_id):
        n = self.find_nodes_by_template({"label": "Player", "template": {"player_id": player_id}})
        if n is not None and len(n) > 0:
            n = n[0]
        else:
            n = None

        return n

    def create_team(self, team_id, team_name):
        n = Node("Team", team_id=team_id, team_name=team_name)
        tx = self._graph.begin(autocommit=True)
        tx.create(n)
        return n

    def get_team(self, team_id):
        n = self.find_nodes_by_template({"label": "Team", "template": {"team_id": team_id}})
        if n is not None and len(n) > 0:
            n = n[0]
        else:
            n = None

        return n

    def create_supports(self, uni, team_id):
        """
        Create a SUPPORTS relationship from a Fan to a Team.
        :param uni: The UNI for a fan.
        :param team_id: An ID for a team.
        :return: The created SUPPORTS relationship from the Fan to the Team
        """
        f = self.get_fan(uni)
        t = self.get_team(team_id)
        r = Relationship(f, "SUPPORTS", t)
        tx = self._graph.begin(autocommit=True)
        tx.create(r)
        return r

    def get_appearance(self, player_id, team_id, year_id):
        """
        Get the information about appearances for a player and team.
        :param player_id: player_id
        :param team_id: team_id
        :param year_id: The year for getting appearances.
        :return:
        """
        try:
            # Get the Nodes at the ends of the relationship representing appearances.
            p = self.get_player(player_id)
            t = self.get_team(team_id)

            # Run a match looking for relationships of a specific type linking the nodes.
            rm = self._graph.match(nodes=[p, t], r_type="APPEARED")
            result = []

            # If there is a list of relationships.
            if rm is not None:
                for r in rm:

                    # The type will be a class APPEARED() because of the OO mapping.
                    node_type = type(r).__name__
                    year = r['year']

                    # If the type and year are correct, add to result
                    if node_type == "APPEARED" and (year == year_id or year_id is None):
                        result.append(r)

                return result
            else:
                return None
        except Exception as e:
            print("get_appearance: Exception e = ", e)
            raise e

    # Create an APPEARED relationship from a player to a Team
    def create_appearance_all(self, player_id, team_id, year, games):
        """

        :param player_id: O
        :param team_id:
        :param year:
        :param games:
        :return:
        """
        try:
            tx = self._graph.begin(autocommit=False)
            q = "match (n:Player {player_id: '" + player_id + "'}), " + \
                "(t:Team {team_id: '" + team_id + "'}) " + \
                "create (n)-[r:APPEARED { games: " + str(games) + ", year : " + str(year) + \
                "}]->(t)"
            result = self._graph.run(q)
            tx.commit()
        except Exception as e:
            print("create_appearances: exception = ", e)

    # Create a FOLLOWS relationship from a Fan to another Fan.
    def create_follows(self, follower, followed):
        f = self.get_fan(follower)
        t = self.get_fan(followed)
        r = Relationship(f, "FOLLOWS", t)
        tx = self._graph.begin(autocommit=True)
        tx.create(r)

    # ----- sh3907 implementation begins----- #
    def get_comment(self, comment_id):
        """

        :param comment_id: Comment ID
        :return: Comment
        """
        # create a matcher
        matcher = NodeMatcher(self._graph)

        # connect fan node to comment node
        comment_n = matcher.match("Comment", comment_id=comment_id).first()

        # return type: Node
        return comment_n

    def create_comment(self, uni, comment, team_id=None, player_id=None):
        """
        Creates a comment
        :param uni: The UNI for the Fan making the comment.
        :param comment: A simple string.
        :param team_id: A valid team ID or None. team_id and player_id cannot BOTH be None.
        :param player_id: A valid player ID or None
        :return: The Node representing the comment.
        """
        try:
            tx = self._graph.begin(autocommit=False)
            # create a random id for comment
            cid = uuid.uuid4()

            # create comment node
            comment_n = Node("Comment", comment=comment, comment_id=str(cid))
            tx.create(comment_n)
            print(comment_n)

            # create a matcher
            matcher = NodeMatcher(self._graph)

            # connect fan node to comment node
            fan_n = matcher.match("Fan", uni=uni).first()
            rela = Relationship(fan_n, "COMMENT_BY", comment_n)
            tx.create(rela)

            # connect comment node to team node
            if team_id:
                team_n = matcher.match("Team", team_id=team_id).first()
                rela = Relationship(comment_n, "COMMENT_ON", team_n)
                tx.create(rela)

            # connect comment node to player node
            if player_id:
                player_n = matcher.match("Player", player_id=player_id).first()
                rela = Relationship(comment_n, "COMMENT_ON", player_n)
                tx.create(rela)

            tx.commit()

        except Exception as e:
            print("create_comment: exception = ", e)

        # return type: Node
        return comment_n

    def create_sub_comment(self, uni, origin_comment_id, comment):
        """
        Create a sub-comment (response to a comment or response) and links with parent in thread.
        :param uni: ID of the Fan making the comment.
        :param origin_comment_id: Id of the comment to which this is a response.
        :param comment: Comment string
        :return: Created comment.
        """
        try:
            tx = self._graph.begin(autocommit=False)

            # create a random id for comment
            cid = uuid.uuid4()

            # create comment node
            comment_n = Node("Comment", comment=comment, comment_id=str(cid))
            tx.create(comment_n)

            # create a matcher
            matcher = NodeMatcher(self._graph)

            # connect fan node to comment node
            fan_n = matcher.match("Fan", uni=uni).first()
            rela = Relationship(fan_n, "COMMENT_BY", comment_n)
            tx.create(rela)

            # connect comment node to original comment node
            origin_comment_n = matcher.match("Comment", comment_id=origin_comment_id).first()
            rela = Relationship(comment_n, "COMMENT_ON", origin_comment_n)
            tx.create(rela)

            tx.commit()

        except Exception as e:
            print("create_comment: exception = ", e)

        # return type: Node
        return comment_n

    def get_sub_comments(self, comment_id):
        """

        :param comment_id: The unique ID of the comment.
        :return: The sub-comments.
        """
        # create a matcher
        matcher = NodeMatcher(self._graph)

        comment_n = matcher.match("Comment", comment_id=comment_id).first()

        sub_comments = RelationshipMatcher(self._graph).match((None, comment_n), "COMMENT_ON")

        # return type: RelationshipMatch
        return sub_comments

    def get_player_comments(self, player_id):
        """
        Gets all of the comments associated with a player, Also returns the Nodes for people making the comments.
        :param player_id: ID of the player.
        :return: Graph containing comment, comment streams and commenters.
        """
        tx = self._graph.begin(autocommit=False)

        q = "MATCH (n:Player {player_id:'" + player_id + "'}) MATCH (c:Comment)-[r:COMMENT_ON]->(n) RETURN c;"

        result = self._graph.run(q)

        tx.commit()

        # return type: Cursor
        return result

    def get_team_comments(self, team_id):
        """
        Gets all of the comments associated with a team.  Also returns the Nodes for people making the comments.
        :param player_id: ID of the team.
        :return: Graph containing comment, comment streams and commenters.
        """
        # create a matcher
        matcher = NodeMatcher(self._graph)

        team_n = matcher.match("Team", team_id=team_id).first()

        comments = RelationshipMatcher(self._graph).match((None, team_n), "COMMENT_ON")

        # return type: RelationshipMatch
        return comments

    def get_players_by_team(self, team_id, yearid):
        """

        :param team_id: The ID of a team.
        :param yearid: A year.
        :return: Returns the players who played for the team in the year.
        """
        tx = self._graph.begin(autocommit=False)

        q = "MATCH (t:Team {team_id:'" + team_id + "'}) MATCH (p:Player)-[r:APPEARED]->(n) WHERE r.year = " + yearid + " RETURN p;"

        result = self._graph.run(q)

        tx.commit()

        # return type: Cursor
        return result
    # ----- sh3907 implementation ends----- #


### 2. Create a Fan graph and connecting with mysql db

In [2]:
import fan_graph as f
import json
import pymysql
import pandas as pd

pd.set_option('display.width', 132)

# connect with mysql database
cnx = pymysql.connect(host='localhost',
                             user='dbuser',
                             password='dbuserdbuser',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# connect with neo4j database
fg = f.FanGraph(auth=('neo4j', 'neo4j'),
                  host="localhost",
                  port=7687,
                  secure=False)
print("fg = ", fg)

fg =  <fan_graph.FanGraph object at 0x0000025CDFDFBF98>


### 3-1. Load players from laman2017.people

In [3]:
def load_players():

    """
    Load a subset of the player data. We only load a subset of the columns and the rows.
    """

    q = "SELECT playerID, nameLast, nameFirst FROM People where  " + \
        "exists (select * from appearances where appearances.playerID = " + \
        " people.playerID and yearID >= 2010)"


    curs = cnx.cursor()
    curs.execute(q)
    
    r = curs.fetchone()
    cnt = 0

    # Loop until we are out of rows.
    while r is not None:
        print(r)
        cnt += 1
        if r is not None:
            p = fg.create_player(player_id=r['playerID'], last_name=r['nameLast'], first_name=r['nameFirst'])
            print("Created player = ", p)

        r = curs.fetchone()

    print("Loaded ", cnt, "records.")


### 3-2. Load teams from laman2017.teams

In [4]:
def load_teams():
    q = "SELECT distinct teamid, name from teams where yearid >= 2000"

    curs = cnx.cursor()
    curs.execute(q)

    cnt = 0
    r = curs.fetchone()
    while r is not None:

        print(r)
        cnt += 1

        p = fg.create_team(team_id=r['teamid'], team_name=r['name'])
        print("Created team = ", p)

        r = curs.fetchone()

    print("Loaded ", cnt, "records.")

### 3-3. Load appearances from laman2017.appearances

In [5]:
def load_appearances():

    q = \
    "SELECT distinct playerid, teamid, yearid, g_all as games " + \
    " from appearances where yearid >= 2010"

    curs = cnx.cursor()
    curs.execute(q)

    r = curs.fetchone()
    cnt = 0
    while r is not None:
        print(r)
        cnt += 1

        if r is not None:
            try:
                p = fg.create_appearance_all(team_id=r['teamid'], player_id=r['playerid'], \
                                        games=r['games'], year=r['yearid'])
                print("Created appearances = ", json.dumps(p))
            except Exception as e:
                print("Could not create.")
        r = curs.fetchone()

    print("Loaded ", cnt, "records.")

#### Execute.

In [6]:
load_players()
load_teams()
load_appearances()

{'playerID': 'aardsda01', 'nameLast': 'Aardsma', 'nameFirst': 'David'}
Created player =  (_36022:Player {first_name: 'David', last_name: 'Aardsma', player_id: 'aardsda01'})
{'playerID': 'abadfe01', 'nameLast': 'Abad', 'nameFirst': 'Fernando'}
Created player =  (_36023:Player {first_name: 'Fernando', last_name: 'Abad', player_id: 'abadfe01'})
{'playerID': 'abreubo01', 'nameLast': 'Abreu', 'nameFirst': 'Bobby'}
Created player =  (_36024:Player {first_name: 'Bobby', last_name: 'Abreu', player_id: 'abreubo01'})
{'playerID': 'abreujo02', 'nameLast': 'Abreu', 'nameFirst': 'Jose'}
Created player =  (_36025:Player {first_name: 'Jose', last_name: 'Abreu', player_id: 'abreujo02'})
{'playerID': 'abreuju01', 'nameLast': 'Abreu', 'nameFirst': 'Juan'}
Created player =  (_36026:Player {first_name: 'Juan', last_name: 'Abreu', player_id: 'abreuju01'})
{'playerID': 'abreuto01', 'nameLast': 'Abreu', 'nameFirst': 'Tony'}
Created player =  (_36027:Player {first_name: 'Tony', last_name: 'Abreu', player_id: 

Created player =  (_39113:Player {first_name: 'Christian', last_name: 'Arroyo', player_id: 'arroych01'})
{'playerID': 'arrueer01', 'nameLast': 'Arruebarrena', 'nameFirst': 'Erisbel'}
Created player =  (_39114:Player {first_name: 'Erisbel', last_name: 'Arruebarrena', player_id: 'arrueer01'})
{'playerID': 'ascanjo01', 'nameLast': 'Ascanio', 'nameFirst': 'Jose'}
Created player =  (_39115:Player {first_name: 'Jose', last_name: 'Ascanio', player_id: 'ascanjo01'})
{'playerID': 'ascheco01', 'nameLast': 'Asche', 'nameFirst': 'Cody'}
Created player =  (_39136:Player {first_name: 'Cody', last_name: 'Asche', player_id: 'ascheco01'})
{'playerID': 'asheral01', 'nameLast': 'Asher', 'nameFirst': 'Alec'}
Created player =  (_39137:Player {first_name: 'Alec', last_name: 'Asher', player_id: 'asheral01'})
{'playerID': 'ashlene01', 'nameLast': 'Ashley', 'nameFirst': 'Nevin'}
Created player =  (_39138:Player {first_name: 'Nevin', last_name: 'Ashley', player_id: 'ashlene01'})
{'playerID': 'astinba01', 'nameL

Created player =  (_39186:Player {first_name: 'Danny', last_name: 'Barnes', player_id: 'barneda02'})
{'playerID': 'barneja01', 'nameLast': 'Barnes', 'nameFirst': 'Jacob'}
Created player =  (_39187:Player {first_name: 'Jacob', last_name: 'Barnes', player_id: 'barneja01'})
{'playerID': 'barnema01', 'nameLast': 'Barnes', 'nameFirst': 'Matt'}
Created player =  (_39188:Player {first_name: 'Matt', last_name: 'Barnes', player_id: 'barnema01'})
{'playerID': 'barnesc01', 'nameLast': 'Barnes', 'nameFirst': 'Scott'}
Created player =  (_39189:Player {first_name: 'Scott', last_name: 'Barnes', player_id: 'barnesc01'})
{'playerID': 'barneto01', 'nameLast': 'Barnette', 'nameFirst': 'Tony'}
Created player =  (_39190:Player {first_name: 'Tony', last_name: 'Barnette', player_id: 'barneto01'})
{'playerID': 'barnhtu01', 'nameLast': 'Barnhart', 'nameFirst': 'Tucker'}
Created player =  (_39191:Player {first_name: 'Tucker', last_name: 'Barnhart', player_id: 'barnhtu01'})
{'playerID': 'baronst01', 'nameLast': 

Created player =  (_39241:Player {first_name: 'Engel', last_name: 'Beltre', player_id: 'beltren01'})
{'playerID': 'beltrom01', 'nameLast': 'Beltre', 'nameFirst': 'Omar'}
Created player =  (_39242:Player {first_name: 'Omar', last_name: 'Beltre', player_id: 'beltrom01'})
{'playerID': 'beninan01', 'nameLast': 'Benintendi', 'nameFirst': 'Andrew'}
Created player =  (_39243:Player {first_name: 'Andrew', last_name: 'Benintendi', player_id: 'beninan01'})
{'playerID': 'benoijo01', 'nameLast': 'Benoit', 'nameFirst': 'Joaquin'}
Created player =  (_39244:Player {first_name: 'Joaquin', last_name: 'Benoit', player_id: 'benoijo01'})
{'playerID': 'bensojo01', 'nameLast': 'Benson', 'nameFirst': 'Joe'}
Created player =  (_39245:Player {first_name: 'Joe', last_name: 'Benson', player_id: 'bensojo01'})
{'playerID': 'bensokr01', 'nameLast': 'Benson', 'nameFirst': 'Kris'}
Created player =  (_39246:Player {first_name: 'Kris', last_name: 'Benson', player_id: 'bensokr01'})
{'playerID': 'beresja01', 'nameLast': 

Created player =  (_39296:Player {first_name: 'Brennan', last_name: 'Boesch', player_id: 'boescbr01'})
{'playerID': 'bogaexa01', 'nameLast': 'Bogaerts', 'nameFirst': 'Xander'}
Created player =  (_39297:Player {first_name: 'Xander', last_name: 'Bogaerts', player_id: 'bogaexa01'})
{'playerID': 'boggsbr01', 'nameLast': 'Boggs', 'nameFirst': 'Brandon'}
Created player =  (_39298:Player {first_name: 'Brandon', last_name: 'Boggs', player_id: 'boggsbr01'})
{'playerID': 'boggsmi01', 'nameLast': 'Boggs', 'nameFirst': 'Mitchell'}
Created player =  (_39299:Player {first_name: 'Mitchell', last_name: 'Boggs', player_id: 'boggsmi01'})
{'playerID': 'bogusbr01', 'nameLast': 'Bogusevic', 'nameFirst': 'Brian'}
Created player =  (_39300:Player {first_name: 'Brian', last_name: 'Bogusevic', player_id: 'bogusbr01'})
{'playerID': 'bolsimi01', 'nameLast': 'Bolsinger', 'nameFirst': 'Mike'}
Created player =  (_39301:Player {first_name: 'Mike', last_name: 'Bolsinger', player_id: 'bolsimi01'})
{'playerID': 'bondej

...

Created appearances =  null
{'playerid': 'weeksje01', 'teamid': 'OAK', 'yearid': '2013', 'games': '8'}
Created appearances =  null
{'playerid': 'weeksje01', 'teamid': 'BAL', 'yearid': '2014', 'games': '3'}
Created appearances =  null
{'playerid': 'weeksje01', 'teamid': 'BOS', 'yearid': '2014', 'games': '14'}
Created appearances =  null
{'playerid': 'weeksje01', 'teamid': 'BOS', 'yearid': '2015', 'games': '3'}
Created appearances =  null
{'playerid': 'weeksje01', 'teamid': 'SDN', 'yearid': '2016', 'games': '17'}
Created appearances =  null
{'playerid': 'weeksri01', 'teamid': 'MIL', 'yearid': '2010', 'games': '160'}
Created appearances =  null
{'playerid': 'weeksri01', 'teamid': 'MIL', 'yearid': '2011', 'games': '118'}
Created appearances =  null
{'playerid': 'weeksri01', 'teamid': 'MIL', 'yearid': '2012', 'games': '157'}
Created appearances =  null
{'playerid': 'weeksri01', 'teamid': 'MIL', 'yearid': '2013', 'games': '104'}
Created appearances =  null
{'playerid': 'weeksri01', 'teamid':

Created appearances =  null
{'playerid': 'wilheto01', 'teamid': 'SEA', 'yearid': '2015', 'games': '53'}
Created appearances =  null
{'playerid': 'wilheto01', 'teamid': 'SEA', 'yearid': '2016', 'games': '29'}
Created appearances =  null
{'playerid': 'wilheto01', 'teamid': 'TEX', 'yearid': '2016', 'games': '21'}
Created appearances =  null
{'playerid': 'wilheto01', 'teamid': 'ARI', 'yearid': '2017', 'games': '27'}
Created appearances =  null
{'playerid': 'wilkad01', 'teamid': 'DET', 'yearid': '2011', 'games': '5'}
Created appearances =  null
{'playerid': 'wilkad01', 'teamid': 'DET', 'yearid': '2012', 'games': '3'}
Created appearances =  null
{'playerid': 'wilkad01', 'teamid': 'LAA', 'yearid': '2015', 'games': '1'}
Created appearances =  null
{'playerid': 'wilkad01', 'teamid': 'MIN', 'yearid': '2017', 'games': '3'}
Created appearances =  null
{'playerid': 'wilkad01', 'teamid': 'NYN', 'yearid': '2017', 'games': '1'}
Created appearances =  null
{'playerid': 'wilkeaa01', 'teamid': 'MIL', 'ye

Created appearances =  null
{'playerid': 'wilsojo03', 'teamid': 'DET', 'yearid': '2015', 'games': '21'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'PIT', 'yearid': '2012', 'games': '8'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'PIT', 'yearid': '2013', 'games': '58'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'PIT', 'yearid': '2014', 'games': '70'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'NYA', 'yearid': '2015', 'games': '74'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'DET', 'yearid': '2016', 'games': '66'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'CHN', 'yearid': '2017', 'games': '23'}
Created appearances =  null
{'playerid': 'wilsoju10', 'teamid': 'DET', 'yearid': '2017', 'games': '42'}
Created appearances =  null
{'playerid': 'wilsomi02', 'teamid': 'SEA', 'yearid': '2011', 'games': '8'}
Created appearances =  null
{'playerid': 'wilsoty01', 'teamid': 'B

Created appearances =  null
{'playerid': 'worthda01', 'teamid': 'DET', 'yearid': '2014', 'games': '20'}
Created appearances =  null
{'playerid': 'worthda01', 'teamid': 'HOU', 'yearid': '2016', 'games': '16'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2010', 'games': '157'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2011', 'games': '102'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2012', 'games': '156'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2013', 'games': '112'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2014', 'games': '134'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2015', 'games': '38'}
Created appearances =  null
{'playerid': 'wrighda03', 'teamid': 'NYN', 'yearid': '2016', 'games': '37'}
Created appearances =  null
{'playerid': 'wrighda04', 'team

Created appearances =  null
{'playerid': 'youngde03', 'teamid': 'PHI', 'yearid': '2013', 'games': '80'}
Created appearances =  null
{'playerid': 'youngde03', 'teamid': 'TBA', 'yearid': '2013', 'games': '23'}
Created appearances =  null
{'playerid': 'youngde03', 'teamid': 'BAL', 'yearid': '2014', 'games': '83'}
Created appearances =  null
{'playerid': 'youngde03', 'teamid': 'BAL', 'yearid': '2015', 'games': '52'}
Created appearances =  null
{'playerid': 'youngde04', 'teamid': 'PIT', 'yearid': '2010', 'games': '110'}
Created appearances =  null
{'playerid': 'younger03', 'teamid': 'COL', 'yearid': '2010', 'games': '51'}
Created appearances =  null
{'playerid': 'younger03', 'teamid': 'COL', 'yearid': '2011', 'games': '77'}
Created appearances =  null
{'playerid': 'younger03', 'teamid': 'COL', 'yearid': '2012', 'games': '98'}
Created appearances =  null
{'playerid': 'younger03', 'teamid': 'COL', 'yearid': '2013', 'games': '57'}
Created appearances =  null
{'playerid': 'younger03', 'teamid':

![player_team](player_team.png)

### 4. Load Fans data

In [7]:
def load_fans():
    fg.create_fan(uni="js1", last_name="Smith", first_name="John")
    fg.create_fan(uni="ja1", last_name="Adams", first_name="John")
    fg.create_fan(uni="tj1", last_name="Jefferson", first_name="Thomas")
    fg.create_fan(uni="gw1", last_name="Washing", first_name="George")
    fg.create_fan(uni="jm1", last_name="Monroe", first_name="James")
    fg.create_fan(uni="al1", last_name="Lincoln", first_name="Abraham")


def load_follows_fans():
    fg.create_follows(follower="gw1", followed="js1")
    fg.create_follows(follower="tj1", followed="gw1")
    fg.create_follows(follower="ja1", followed="gw1")
    fg.create_follows(follower="jm1", followed="gw1")
    fg.create_follows(follower="al1", followed="jm1")


def create_supports():
    fg.create_supports("gw1", "WAS")
    fg.create_supports("ja1", "BOS")
    fg.create_supports("tj1", "WAS")
    fg.create_supports("jm1", "NYA")
    fg.create_supports("al1", "CHA")
    fg.create_supports("al1", "CHN")

#### Execute.

In [8]:
load_fans()
load_follows_fans()
create_supports()

![fans1](fans1.png)

#### Change display style

![fans2](fans2.png)

### 5. Testing function implementations of FanGraph class


#### Connection setting

In [9]:
import fan_graph as f
import json
import pymysql

# connect with mysql database
cnx = pymysql.connect(host='localhost',
                             user='dbuser',
                             password='dbuserdbuser',
                             db='lahman2017',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

# connect with neo4j database
fg = f.FanGraph(auth=('neo4j', 'neo4j'),
                  host="localhost",
                  port=7687,
                  secure=False)
print("fg = ", fg)

fg =  <fan_graph.FanGraph object at 0x0000025CDFC93898>


#### Helper function for displaying comment node.

In [10]:
import pandas as pd

pd.set_option('display.width', 132)

def betterDisplay(c):
    result = {}
    for r in c:
        s = r.start_node
        e = r.end_node
        cid = s['comment_id']
        this_c = result.get(cid, None)
        if this_c is None:
            this_c = {}
            this_c['comment'] = s['comment']

        if "Team" in e.labels:
            this_c['team'] = e['team_name']
        elif "Player" in e.labels:
            this_c['player'] = e['last_name'] + "," + e['first_name']

        result[cid] = this_c

    final_result = []
    for k, v in result.items():
        new_r = {}
        new_r["id"] = k
        new_r = {**new_r, **v}
        final_result.append(new_r)

    df2 = pd.DataFrame(final_result)
    print(df2)

### 5-1. create_comment(self, uni, comment, team_id=None, player_id=None)

In [11]:
comment_n = fg.create_comment("ja1", "Awesome", team_id="BOS", player_id="pedrodu01")
cid = comment_n['comment_id']

(_42133:Comment {comment: 'Awesome', comment_id: 'f8074b43-271f-4fde-ae29-716207f76fde'})


![comment](comment1.png)

### 5-2. get_comment(self, comment_id)

In [12]:
comment_n = fg.get_comment(cid)
print(comment_n)
print(comment_n['comment'])

(_42133:Comment {comment: 'Awesome', comment_id: 'f8074b43-271f-4fde-ae29-716207f76fde'})
Awesome


### 5-3. create_sub_comment(self, uni, origin_comment_id, comment)

In [13]:
sub_comment_n = fg.create_sub_comment("tj1", cid, "I don't agree, they are so so.")
sub_cid = sub_comment_n['comment_id']
print(sub_comment_n)
print(sub_comment_n['comment'])

(_29811:Comment {comment: "I don't agree, they are so so.", comment_id: 'cbf5ca6c-2202-4d87-ad0e-140379b1ed4e'})
I don't agree, they are so so.


![sub-comment](sub_comment.png)

### 5-4. get_sub_comments(self, comment_id)

In [14]:
comment_rm = fg.get_sub_comments(cid) # get cid's sub comments, return type: RelationshipMatch
print(comment_rm)
print(betterDisplay(comment_rm))

                          comment                                    id
0  I don't agree, they are so so.  cbf5ca6c-2202-4d87-ad0e-140379b1ed4e
None


### 5-5. get_player_comments(self, player_id)

In [15]:
# create another comment
fg.create_comment("gw1", "Totally", "BOS", "pedrodu01")

comment_ns = fg.get_player_comments('pedrodu01')
for comment in comment_ns:
    print(comment)

(_29812:Comment {comment: 'Totally', comment_id: 'db8d18db-0d5c-43ff-8a29-100b5e777658'})
<Record c=(_29812:Comment {comment: 'Totally', comment_id: 'db8d18db-0d5c-43ff-8a29-100b5e777658'})>
<Record c=(_42133:Comment {comment: 'Awesome', comment_id: 'f8074b43-271f-4fde-ae29-716207f76fde'})>


![comment2](comment2.png)

### 5-6. get_team_comments(self, team_id)

In [17]:
comment_rm = fg.get_team_comments('BOS')
print(comment_ns)
print(betterDisplay(comment_rm))

   comment                                    id            team
0  Totally  db8d18db-0d5c-43ff-8a29-100b5e777658  Boston Red Sox
1  Awesome  f8074b43-271f-4fde-ae29-716207f76fde  Boston Red Sox
None


### 5-7. get_players_by_team(self, team_id, yearid)

In [18]:
players = fg.get_players_by_team('BOS', '2013') # return a cursor instance
for player in players:
    print(player)

<Record p=(_36022:Player {first_name: 'David', last_name: 'Aardsma', player_id: 'aardsda01'})>
<Record p=(_36023:Player {first_name: 'Fernando', last_name: 'Abad', player_id: 'abadfe01'})>
<Record p=(_36027:Player {first_name: 'Tony', last_name: 'Abreu', player_id: 'abreuto01'})>
<Record p=(_36029:Player {first_name: 'Alfredo', last_name: 'Aceves', player_id: 'aceveal01'})>
<Record p=(_36031:Player {first_name: 'Dustin', last_name: 'Ackley', player_id: 'ackledu01'})>
<Record p=(_39019:Player {first_name: 'David', last_name: 'Adams', player_id: 'adamsda02'})>
<Record p=(_39021:Player {first_name: 'Matt', last_name: 'Adams', player_id: 'adamsma01'})>
<Record p=(_39022:Player {first_name: 'Mike', last_name: 'Adams', player_id: 'adamsmi03'})>
<Record p=(_39025:Player {first_name: 'Jim', last_name: 'Adduci', player_id: 'adducji02'})>
<Record p=(_39027:Player {first_name: 'Ehire', last_name: 'Adrianza', player_id: 'adriaeh01'})>
<Record p=(_39028:Player {first_name: 'Jeremy', last_name: 'Aff

<Record p=(_40877:Player {first_name: 'Mike', last_name: 'Minor', player_id: 'minormi01'})>
<Record p=(_40891:Player {first_name: 'Jose', last_name: 'Molina', player_id: 'molinjo01'})>
<Record p=(_40891:Player {first_name: 'Jose', last_name: 'Molina', player_id: 'molinjo01'})>
<Record p=(_40892:Player {first_name: 'Yadier', last_name: 'Molina', player_id: 'molinya01'})>
<Record p=(_40899:Player {first_name: 'Johnny', last_name: 'Monell', player_id: 'moneljo01'})>
<Record p=(_40902:Player {first_name: 'Jesus', last_name: 'Montero', player_id: 'monteje01'})>
<Record p=(_40903:Player {first_name: 'Miguel', last_name: 'Montero', player_id: 'montemi01'})>
<Record p=(_40907:Player {first_name: 'Luke', last_name: 'Montz', player_id: 'montzlu01'})>
<Record p=(_40908:Player {first_name: 'Adam', last_name: 'Moore', player_id: 'mooread01'})>
<Record p=(_40911:Player {first_name: 'Matt', last_name: 'Moore', player_id: 'moorema02'})>
<Record p=(_40911:Player {first_name: 'Matt', last_name: 'Moore',

#### NOTE
#### delete all nodes in neo4j

MATCH (n) DETACH DELETE n;

### Part 2- Redis

#### Main functions

In [19]:
import redis
import json
from redis_helper import MysqlHelpers
from urllib.parse import urlencode
"""
Connect to local Redis server. StrictRedis complies more closely with standard than
simple Redis client in this package. decode_responses specifies whether or not to convert
bytes to UTF8 character string or treat as raw bytes, e.g. an image, audio stream, etc.
"""


class RedisFunctions:

    def __init__(self):
        self.r = redis.StrictRedis(
            host='localhost',
            port=6379,
            charset="utf-8", decode_responses=True)

        if self.r:
            print("Connected with Redis.")

        self.db = MysqlHelpers()

    # ----- sh3907 implementation begins----- #
    # requirement function 1   
    def retrieve_by_template(self, table, template, fields=None, limit=None, offset=None, order_by=None, use_cache=False):

        in_cache = None

        if use_cache:

            in_cache = self.check_cache(table, template, fields, limit, offset, order_by)

            if in_cache:

                print("Check cache returned: ", json.dumps(json.loads(in_cache), indent=2))
                print("CACHE HIT")

        if not use_cache or not in_cache:
            q_result = self.db.find_by_template(table, template, fields, limit, offset, order_by)

            if q_result:
                print("Retrieve data from mysql database: ", json.dumps(q_result, indent=2))
                self.add_to_cache(table, template, fields, limit, offset, order_by, q_result)
                
                if use_cache:
                    print("CACHE MISS")
            else:
                print("Data not found from the database. Please modify your input")
    
    
    # requirement function 2      
    def retrieve_from_cache(self, key):
        """
        :param key: A valid Redis key.
        :return: The "map object" associated with the key.
        """
        result = self.r.get(key)
        return result
    
    
    # requirement function 3
    def add_to_cache(self, table, tmp, fields, limit, offset, order_by, q_result):
        key = self.generate_key(table, tmp, fields, limit, offset, order_by)
        v = json.dumps(q_result)
        save_result = self.r.set(key, v)

        if save_result:
            print("Successful add key={} data into cache".format(key))
        else:
            print("Fail to add into cache.")

            
    def generate_key(self, table, template, fields, limit, offset, order_by):
        tmp = template.copy()
        if fields:
            tmp['field'] = ",".join(fields)
        if limit:
            tmp['limit'] = limit
        if offset:
            tmp['offset'] = offset
        if order_by:
            tmp['order_by'] = order_by
        k = urlencode(tmp)
        k = table + "/" + k

        return k


    def check_cache(self, table, tmp, fields, limit, offset, order_by):
        key = self.generate_key(table, tmp, fields, limit, offset, order_by)
        result = self.retrieve_from_cache(key)
        if result:
            return result
        else:
            return None


    def delete_keys(self):
        keys = self.get_keys()
        self.r.delete(*keys)
# ----- sh3907 implementation ends----- #

    def get_keys(self):
        result = self.r.keys()
        return result


#### Helper functions

In [20]:
'''
HW4 will add checking the Redis cache to a simple, standalone  𝑟𝑒𝑡𝑟𝑖𝑒𝑣𝑒_𝑏𝑦_𝑡𝑒𝑚𝑝𝑙𝑎𝑡𝑒()  function.
Write a function  𝑎𝑑𝑑_𝑡𝑜_𝑐𝑎𝑐ℎ𝑒()  that adds a retrieve result to the cache.
Write a function  𝑟𝑒𝑡𝑟𝑖𝑒𝑣𝑒_𝑓𝑟𝑜𝑚_𝑐𝑎𝑐ℎ𝑒()  that checks the cache and returns the value if present.
Modify  𝑟𝑒𝑡𝑟𝑖𝑒𝑣𝑒_𝑏𝑦_𝑡𝑒𝑚𝑝𝑙𝑎𝑡𝑒()  to use the cache.
Check and return if cached.
Call DB and add to cache if not cached.
'''

import pymysql.cursors

class MysqlHelpers:

    def __init__(self):
        self.db_schema = None                                # Schema containing accessed data
        self.cnx = None                                      # DB connection to use for accessing the data.
        self.key_delimiter = '_'                             # This should probably be a config option.
        self.set_config()


    def get_new_connection(self, params=None):
        if not params:
            params = self.default_db_params

        cnx = pymysql.connect(
            host=params["dbhost"],
            port=params["port"],
            user=params["dbuser"],
            password=params["dbpw"],
            db=params["dbname"],
            charset=params["charset"],
            cursorclass=params["cursorClass"])
        return cnx

    def set_config(self):
        """
        Creates the DB connection and sets the global variables.

        :param cfg: Application configuration data.
        :return: None
        """



        db_params = {
            "dbhost": "localhost",
            "port": 3306,
            "dbname": "lahman2017",
            "dbuser": "dbuser",
            "dbpw": "dbuserdbuser",
            "cursorClass": pymysql.cursors.DictCursor,
            "charset": 'utf8mb4'
        }

        self.db_schema = "lahman2017"

        self.cnx = self.get_new_connection(db_params)

        print("Mysql Connection Object: ", self.cnx)
        return self.cnx


    # Given one of our magic templates, forms a WHERE clause.
    # { a: b, c: d } --> WHERE a=b and c=d. Currently treats everything as a string.
    # We can fix this by using PyMySQL connector query templates.
    def templateToWhereClause(self, t):
        s = ""
        for k,v in t.items():
            if s != "":
                s += " AND "
            s += k + "=\'" + v + "\'"

        if s != "":
            s = "WHERE " + s + ";"

        return s


    def run_q(self, cnx, q, args, fetch=False, commit=True):
        """
        :param cnx: The database connection to use.
        :param q: The query string to run.
        :param args: Parameters to insert into query template if q is a template.
        :param fetch: True if this query produces a result and the function should perform and return fetchall()
        :return:
        """
        #debug_message("run_q: q = " + q)
        #ut.debug_message("Q = " + q)
        #ut.debug_message("Args = ", args)

        result = None

        try:
            cursor = cnx.cursor()
            result = cursor.execute(q, args)
            if fetch:
                result = cursor.fetchall()
            if commit:
                cnx.commit()
        except Exception as original_e:
            #print("dffutils.run_q got exception = ", original_e)
            raise(original_e)

        return result

    def find_by_template(self, table, t, fields=None, limit=None, offset=None, orderBy=None):

        if t is not None:
            w = self.templateToWhereClause(t)
        else:
            w = ""

        if orderBy is not None:
            o = "order by " + ",".join(orderBy['fields']) + " " + orderBy['direction'] + " "
        else:
            o = ""

        if limit is not None:
            w += " LIMIT " + str(limit)
        if offset is not None:
            w += " OFFSET " + str(offset)

        if fields is None:
            fields = " * "
        else:
            fields = " " + ",".join(fields) + " "

        #cursor = self.cnx.cursor()
        q = "SELECT " + fields + " FROM " + table + " " + w + ";"

        r = self.run_q(self.cnx, q, None, fetch=True, commit=True)

        return r

### Testing

In [21]:
r = RedisFunctions()

Connected with Redis.
Mysql Connection Object:  <pymysql.connections.Connection object at 0x0000025CF28D8550>


##### Retrieve_by_template


In [23]:
tmp = { "nameLast": "Williams", "birthState": "CA"}
fields = ["playerID", "nameLast", "nameFirst", "throws"]
resource = "People"

r.retrieve_by_template(table=resource, template=tmp, fields=fields, use_cache=True)

Retrieve data from mysql database:  [
  {
    "playerID": "willibe01",
    "nameLast": "Williams",
    "nameFirst": "Bernie",
    "throws": "R"
  },
  {
    "playerID": "willido02",
    "nameLast": "Williams",
    "nameFirst": "Don",
    "throws": "R"
  },
  {
    "playerID": "williji03",
    "nameLast": "Williams",
    "nameFirst": "Jimy",
    "throws": "R"
  },
  {
    "playerID": "willike02",
    "nameLast": "Williams",
    "nameFirst": "Ken",
    "throws": "R"
  },
  {
    "playerID": "willima04",
    "nameLast": "Williams",
    "nameFirst": "Matt",
    "throws": "R"
  },
  {
    "playerID": "willimi02",
    "nameLast": "Williams",
    "nameFirst": "Mitch",
    "throws": "L"
  },
  {
    "playerID": "williri02",
    "nameLast": "Williams",
    "nameFirst": "Rinaldo",
    "throws": "R"
  },
  {
    "playerID": "williri03",
    "nameLast": "Williams",
    "nameFirst": "Rick",
    "throws": "R"
  },
  {
    "playerID": "willish01",
    "nameLast": "Williams",
    "nameFirst": "Shad",


##### Retrieve same template again

In [24]:
r.retrieve_by_template(table=resource, template=tmp, fields=fields, use_cache=True)

Check cache returned:  [
  {
    "playerID": "willibe01",
    "nameLast": "Williams",
    "nameFirst": "Bernie",
    "throws": "R"
  },
  {
    "playerID": "willido02",
    "nameLast": "Williams",
    "nameFirst": "Don",
    "throws": "R"
  },
  {
    "playerID": "williji03",
    "nameLast": "Williams",
    "nameFirst": "Jimy",
    "throws": "R"
  },
  {
    "playerID": "willike02",
    "nameLast": "Williams",
    "nameFirst": "Ken",
    "throws": "R"
  },
  {
    "playerID": "willima04",
    "nameLast": "Williams",
    "nameFirst": "Matt",
    "throws": "R"
  },
  {
    "playerID": "willimi02",
    "nameLast": "Williams",
    "nameFirst": "Mitch",
    "throws": "L"
  },
  {
    "playerID": "williri02",
    "nameLast": "Williams",
    "nameFirst": "Rinaldo",
    "throws": "R"
  },
  {
    "playerID": "williri03",
    "nameLast": "Williams",
    "nameFirst": "Rick",
    "throws": "R"
  },
  {
    "playerID": "willish01",
    "nameLast": "Williams",
    "nameFirst": "Shad",
    "throws":

##### Retrieve other template & not using cache

In [25]:
tmp = { "nameLast": "Avery", "birthState": "GA"}
fields = ["playerID", "nameLast", "nameFirst", "throws"]
resource = "People"

r.retrieve_by_template(table=resource, template=tmp, fields=fields, use_cache=False)

Retrieve data from mysql database:  [
  {
    "playerID": "averyxa01",
    "nameLast": "Avery",
    "nameFirst": "Xavier",
    "throws": "L"
  }
]
Successful add key=People/nameLast=Avery&birthState=GA&field=playerID%2CnameLast%2CnameFirst%2Cthrows data into cache


##### Get all keys in Redis DB

In [26]:
keys = r.get_keys()
print(keys)

['People/nameLast=Avery&birthState=GA&field=playerID%2CnameLast%2CnameFirst%2Cthrows', 'People/nameLast=Williams&birthState=CA&field=playerID%2CnameLast%2CnameFirst%2Cthrows']


##### Use key to retrieve from cache

In [27]:
result = r.retrieve_from_cache(keys[1])
print(json.dumps(json.loads(result), indent=2))

[
  {
    "playerID": "willibe01",
    "nameLast": "Williams",
    "nameFirst": "Bernie",
    "throws": "R"
  },
  {
    "playerID": "willido02",
    "nameLast": "Williams",
    "nameFirst": "Don",
    "throws": "R"
  },
  {
    "playerID": "williji03",
    "nameLast": "Williams",
    "nameFirst": "Jimy",
    "throws": "R"
  },
  {
    "playerID": "willike02",
    "nameLast": "Williams",
    "nameFirst": "Ken",
    "throws": "R"
  },
  {
    "playerID": "willima04",
    "nameLast": "Williams",
    "nameFirst": "Matt",
    "throws": "R"
  },
  {
    "playerID": "willimi02",
    "nameLast": "Williams",
    "nameFirst": "Mitch",
    "throws": "L"
  },
  {
    "playerID": "williri02",
    "nameLast": "Williams",
    "nameFirst": "Rinaldo",
    "throws": "R"
  },
  {
    "playerID": "williri03",
    "nameLast": "Williams",
    "nameFirst": "Rick",
    "throws": "R"
  },
  {
    "playerID": "willish01",
    "nameLast": "Williams",
    "nameFirst": "Shad",
    "throws": "R"
  },
  {
    "play

##### Delete all keys

In [28]:
r.delete_keys()

In [29]:
print(r.get_keys())

[]
